# Machine Learning

## Importar librerias

In [1]:
import pandas as pd
import numpy as np
import gensim
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
#pip install fuzzywuzzy
#pip install python-Levenshtein
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


## Datos limpio

In [262]:
# leer los datos que fueron previamente analizados en el EDA
df = pd.read_parquet('/content/drive/MyDrive/Colab Notebooks/PI1/ETL - EDA/movies_clean.parquet')

In [263]:
# Mostrar información del número de columnas, filas, tipos de datos y valores faltantes presentes en "df"
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45346 entries, 0 to 45345
Data columns (total 22 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   budget                45346 non-null  float64       
 1   genres                42962 non-null  object        
 2   id                    45346 non-null  int64         
 3   original_language     45335 non-null  object        
 4   overview              45346 non-null  object        
 5   popularity            45346 non-null  float64       
 6   production_companies  33557 non-null  object        
 7   production_countries  39138 non-null  object        
 8   release_date          45346 non-null  datetime64[ns]
 9   revenue               45346 non-null  float64       
 10  runtime               45100 non-null  float64       
 11  status                45266 non-null  object        
 12  tagline               20383 non-null  object        
 13  title           

In [264]:
# Seleccionamos solo las filas duplicadas en la columna 'title'
duplicados = df[df.duplicated(['title'], keep=False)]

# Ordenamos el DataFrame por el valor de la columna 'title'
duplicados = duplicados.sort_values('title')

duplicados[['title', 'vote_count', 'release_year', 'production_companies']]

,title,vote_count,release_year,production_companies
24374,10 Minutes,7.0,2002,None
42590,10 Minutes,2.0,2014,None
1159,12 Angry Men,2130.0,1957,"United Artists, Orion-Nova Productions"
15187,12 Angry Men,59.0,1997,MGM Television
32730,12 Chairs,24.0,1971,Mosfilm
...,...,...,...,...
11638,Zodiac,2080.0,2007,"Paramount Pictures, Warner Bros., Phoenix Pict..."
11163,Zoom,140.0,2006,"Columbia Pictures, Revolution Studios"
37819,Zoom,25.0,2015,"Rhombus Media, O2 Filmes"
5756,Zulu,137.0,1964,Diamond Films


In [265]:
df.columns

Index(['budget', 'genres', 'id', 'original_language', 'overview', 'popularity',
       'production_companies', 'production_countries', 'release_date',
       'revenue', 'runtime', 'status', 'tagline', 'title', 'vote_average',
       'vote_count', 'languages', 'director', 'actores', 'overview_clean',
       'release_year', 'return'],
      dtype='object')

# Modelo de recomendación de película basado en contenido

## Feature engineering

In [266]:
# Tokenizar el texto en la columna 'overview'
df['tokens'] = df['overview_clean'].apply(word_tokenize)
# Crear una lista de stopwords en inglés
stop_words = set(stopwords.words('english'))

In [267]:
# Filtrar las stopwords y convertir el texto en minúsculas
df['keywords'] = df['tokens'].apply(lambda tokens: [token.lower() for token in tokens if token.lower() not in stop_words])

In [268]:
# Convertir la lista de palabras clave en una cadena separada por comas
df['keywords'] = df['keywords'].apply(lambda keywords: ', '.join(keywords))

In [269]:
df['keywords'].head()

,keywords
0,"led, woody, andys, toy, live, happily, room, a..."
1,"sibling, judy, peter, discover, enchanted, boa..."
2,"family, wedding, reignites, ancient, feud, nex..."
3,"cheated, mistreated, stepped, woman, holding, ..."
4,"george, bank, ha, recovered, daughter, wedding..."


## Modelos

### Modelo 1

title y overview

In [270]:
# Seleccionando sólo las columnas necesarias para el modelo
modelo1 = df[['title', 'overview_clean']].copy()

In [271]:
# Se crea una instancia de la clase TfidfVectorizer con los parámetros deseados
tfidf_1 = TfidfVectorizer(stop_words="english", ngram_range = (1, 2))
# Aplicar la transformación TF-IDF al texto contenido en la columna "overview" del dataframe 'modelo1'
tfidf_matriz_1 = tfidf_1.fit_transform(modelo1['overview_clean'])

In [272]:
# Cantidad de registros y columnas en tfidf_matriz_1
tfidf_matriz_1.shape

(45346, 1015194)

In [273]:
def get_recomendacion_m1(titulo):
  # Se crea un objeto 'indices' que mapea los títulos de las películas a sus índices correspondientes en el DataFrame 'modelo1'
  indices = pd.Series(modelo1.index, index= modelo1['title']).drop_duplicates()
  # Se busca el índice de la película de entrada utilizando el objeto 'indices'
  idx = pd.Series(indices[titulo]) if titulo in indices else None
  # Si la película de entrada no se encuentra en el DataFrame, se devuelve un mensaje de error
  if idx is None:
    return "Película no encontrada"
  # Si el título de la película está duplicado, se devuelve el índice de la primera aparición del título en el DataFrame
  if modelo1.duplicated(['title']).any():
    primer_idx = modelo1[modelo1['title'] == titulo].index[0]
    if not idx.equals(pd.Series(primer_idx)):
      idx = pd.Series(primer_idx)
  # Se calcula la similitud coseno entre la película de entrada y todas las demás películas en la matriz de características
  simil = sorted(enumerate(cosine_similarity(tfidf_matriz_1[idx], tfidf_matriz_1).flatten()), key=lambda x: x[1], reverse=True)[1:6]
  # Se obtienen los títulos de las películas más similares utilizando el índice de cada película
  recomendaciones = modelo1.iloc[[i[0] for i in simil], :]['title'].tolist()
  # Se devuelve la lista de títulos de las películas recomendadas
  return recomendaciones

In [274]:
# Prueba #1
get_recomendacion_m1('The Dark Knight Rises')

['The Dark Knight',
 'Batman Forever',
 'Batman',
 'Batman Returns',
 'Batman: Under the Red Hood']

In [275]:
# Prueba #2
get_recomendacion_m1('The Avengers')

['The Work and the Glory',
 "Sir Arne's Treasure",
 'Monty Python and the Holy Grail',
 'A Beast at Bay',
 'Enigma']

In [276]:
# Prueba #3 (Película que ha sido lanzada varias veces bajo el mismo nombre)
get_recomendacion_m1('Wonder Woman')

['Wide Eyed and Legless',
 'Trevor Noah: The Daywalker',
 'The Machinist',
 'Diana',
 'Our Dancing Daughters']

In [277]:
# Prueba #4 (Película que ha sido lanzada varias veces bajo el mismo nombre)
get_recomendacion_m1('Titanic')

['Grantham and Rose',
 'Take Care of the Women!',
 'The Legend of 1900',
 'Raise the Titanic',
 'Beyond the Poseidon Adventure']

In [278]:
# Prueba #5
get_recomendacion_m1('Star Wars')

['The Empire Strikes Back',
 'The Star Wars Holiday Special',
 'Star Wars: The Force Awakens',
 'Return of the Jedi',
 'Samson and the Seven Miracles of the World']

In [279]:
# Prueba #6
get_recomendacion_m1('The Hunger Games')

['The Hunger Games: Catching Fire',
 'The Hunger Games: Mockingjay - Part 2',
 'The Starving Games',
 'The Hungover Games',
 'Indie Game: The Movie']

In [280]:
modelo1 = get_recomendacion_m1('The Hunger Games')

### Modelo 2

Title y keywords

In [281]:
# Seleccionando sólo las columnas necesarias para el modelo
modelo2 = df[['title', 'keywords']].copy()
# Se realiza un preprocesamiento en la columna "keywords" para separar los géneros y convertirlos en palabras individuales
modelo2['keywords'] = modelo2['keywords'].fillna('').apply(lambda x: ' '.join(x.replace(',', ' ').replace('-', '').lower().split()))

In [282]:
# Se crea una instancia de la clase TfidfVectorizer con los parámetros deseados
tfidf_2 = TfidfVectorizer(stop_words="english", ngram_range = (1, 2))
# Aplicar la transformación TF-IDF al texto contenido en la columna "overview" del dataframe 'modelo2'
tfidf_matriz_2 = tfidf_2.fit_transform(modelo2['keywords'])

In [283]:
# Cantidad de registros y columnas en tfidf_matriz_2
tfidf_matriz_2.shape

(45346, 1013064)

In [284]:
def get_recomendacion_m2(titulo):
  # Se crea un objeto 'indices' que mapea los títulos de las películas a sus índices correspondientes en el DataFrame 'modelo2'
  indices = pd.Series(modelo2.index, index=modelo2['title']).drop_duplicates()
  # Se busca el índice de la película de entrada utilizando el objeto 'indices'
  idx = pd.Series(indices[titulo]) if titulo in indices else None
  # Si la película de entrada no se encuentra en el DataFrame, se devuelve un mensaje de error
  if idx is None:
    return "Película no encontrada"
  # Si el título de la película está duplicado, se devuelve el índice de la primera aparición del título en el DataFrame
  if modelo2.duplicated(['title']).any():
    primer_idx = modelo2[modelo2['title'] == titulo].index[0]
    if not idx.equals(pd.Series(primer_idx)):
      idx = pd.Series(primer_idx)
  # Se calcula la similitud coseno entre la película de entrada y todas las demás películas en la matriz de características
  simil = sorted(enumerate(cosine_similarity(tfidf_matriz_2[idx], tfidf_matriz_2).flatten()), key=lambda x: x[1], reverse=True)[1:6]
  # Se obtienen los títulos de las películas más similares utilizando el índice de cada película
  recomendaciones = modelo2.iloc[[i[0] for i in simil], :]['title'].tolist()
  # Se devuelve la lista de títulos de las películas recomendadas
  return recomendaciones

In [285]:
# Prueba #1
get_recomendacion_m2('The Dark Knight Rises')

['The Dark Knight',
 'Batman Forever',
 'Batman',
 'Batman Returns',
 'Batman: Under the Red Hood']

In [286]:
# Prueba #2
get_recomendacion_m2('The Avengers')

['The Work and the Glory',
 'Monty Python and the Holy Grail',
 "Sir Arne's Treasure",
 'A Beast at Bay',
 'Enigma']

In [287]:
# Prueba #3 (Película que ha sido lanzada varias veces bajo el mismo nombre)
get_recomendacion_m2('Wonder Woman')

['Wide Eyed and Legless',
 'Trevor Noah: The Daywalker',
 'The Machinist',
 'Diana',
 'Our Dancing Daughters']

In [288]:
# Prueba #4 (Película que ha sido lanzada varias veces bajo el mismo nombre)
get_recomendacion_m2('Titanic')

['Grantham and Rose',
 'Take Care of the Women!',
 'The Legend of 1900',
 'Raise the Titanic',
 'Beyond the Poseidon Adventure']

In [289]:
# Prueba #5
get_recomendacion_m2('Star Wars')

['The Empire Strikes Back',
 'The Star Wars Holiday Special',
 'Star Wars: The Force Awakens',
 'Return of the Jedi',
 'Samson and the Seven Miracles of the World']

In [290]:
# Prueba #6
get_recomendacion_m2('The Hunger Games')

['The Hunger Games: Catching Fire',
 'The Hunger Games: Mockingjay - Part 2',
 'The Starving Games',
 'The Hungover Games',
 'Indie Game: The Movie']

In [291]:
modelo2 = get_recomendacion_m2('The Hunger Games')

### Modelo 3

title, overview y genres

In [292]:
# Seleccionando sólo las columnas necesarias para el modelo
modelo3 = df[['title', 'overview_clean', 'genres']].copy()
# Se realiza un preprocesamiento en la columna "genres" para separar los géneros y convertirlos en palabras individuales
modelo3['genres'] = modelo3['genres'].fillna('').apply(lambda x: ' '.join(x.replace(',', ' ').replace('-', '').lower().split()))

In [293]:
# Se crea una instancia de la clase TfidfVectorizer con los parámetros deseados
tfidf_3 = TfidfVectorizer(stop_words="english", ngram_range=(1, 2))
# Incluir la columna 'vote_count' en la concatenación de columnas
modelo3['concatenado'] = modelo3['overview_clean'] + ' ' + modelo3['genres']
# Aplicar la transformación TF-IDF al texto contenido en las columnas "overview_clean" y "genres" del dataframe 'modelo3'
tfidf_matriz_3 = tfidf_3.fit_transform(modelo3['concatenado'])

In [294]:
# Cantidad de registros y columnas en tfidf_matriz_3
tfidf_matriz_3.shape

(45346, 1035287)

In [295]:
def get_recomendacion_m3(titulo):
  # Se crea un objeto 'indices' que mapea los títulos de las películas a sus índices correspondientes en el DataFrame 'modelo3'
  indices = pd.Series(modelo3.index, index=modelo3['title']).drop_duplicates()
  # Se busca el índice de la película de entrada utilizando el objeto 'indices'
  idx = pd.Series(indices[titulo]) if titulo in indices else None
  # Si la película de entrada no se encuentra en el DataFrame, se devuelve un mensaje
  if idx is None:
    return "Película no encontrada"
  # Si el título de la película está duplicado, se devuelve el índice de la primera aparición del título en el DataFrame
  if modelo3.duplicated(['title']).any():
    primer_idx = modelo3[modelo3['title'] == titulo].index[0]
    if not idx.equals(pd.Series(primer_idx)):
      idx = pd.Series(primer_idx)
  # Se calcula la similitud coseno entre la película de entrada y todas las demás películas en la matriz de características
  simil = sorted(enumerate(cosine_similarity(tfidf_matriz_3[idx], tfidf_matriz_3).flatten()), key=lambda x: x[1], reverse=True)[1:6]
  # Se obtienen los títulos de las películas más similares utilizando el índice de cada película
  recomendaciones = modelo3.iloc[[i[0] for i in simil], :]['title'].tolist()
  # Se devuelve la lista de títulos de las películas recomendadas
  return recomendaciones

In [296]:
# Prueba #1
get_recomendacion_m3('The Dark Knight Rises')

['The Dark Knight',
 'Batman Forever',
 'Batman',
 'Batman Returns',
 'Batman: Under the Red Hood']

In [297]:
# Prueba #2
get_recomendacion_m3('The Avengers')

['The Work and the Glory',
 "Sir Arne's Treasure",
 'Monty Python and the Holy Grail',
 'A Beast at Bay',
 'Enigma']

In [298]:
# Prueba #3 (Película que ha sido lanzada varias veces bajo el mismo nombre)
get_recomendacion_m3('Wonder Woman')

['Wide Eyed and Legless',
 'Trevor Noah: The Daywalker',
 'The Machinist',
 'Diana',
 'Our Dancing Daughters']

In [299]:
# Prueba #4 (Película que ha sido lanzada varias veces bajo el mismo nombre)
get_recomendacion_m3('Titanic')

['Grantham and Rose',
 'Titanic',
 'Take Care of the Women!',
 'Raise the Titanic',
 'The Legend of 1900']

In [300]:
# Prueba #5
get_recomendacion_m3('Star Wars')

['The Empire Strikes Back',
 'The Star Wars Holiday Special',
 'Return of the Jedi',
 'Star Wars: The Force Awakens',
 'Samson and the Seven Miracles of the World']

In [301]:
# Prueba #6
get_recomendacion_m3('The Hunger Games')

['The Hunger Games: Catching Fire',
 'The Hunger Games: Mockingjay - Part 2',
 'The Starving Games',
 'The Hungover Games',
 'The Hunger Games: Mockingjay - Part 1']

In [302]:
modelo3 = get_recomendacion_m3('The Hunger Games')

### Modelo 4

title, overview, genres, director

In [303]:
# Seleccionando sólo las columnas necesarias para el modelo
modelo4 = df[['title', 'overview_clean', 'genres', 'director']].copy()
# Se realiza un preprocesamiento en la columna "genres" para separar los géneros y convertirlos en palabras individuales
modelo4['genres'] = modelo4['genres'].fillna('').apply(lambda x: ' '.join(x.replace(',', ' ').replace('-', '').lower().split()))
# Se realiza un preprocesamiento en la columna "director" para separar los géneros y convertirlos en palabras individuales
modelo4['director'] = modelo4['director'].fillna('').apply(lambda x: ' '.join(x.replace(',', ' ').replace('-', '').lower().split()))

In [304]:
# Se crea una instancia de la clase TfidfVectorizer con los parámetros deseados
tfidf_4 = TfidfVectorizer(stop_words="english", ngram_range=(1, 2))
# Aplicar la transformación TF-IDF al texto contenido en las columnas "overview_clean", "genres" y "director" del dataframe 'modelo4'
tfidf_matriz_4 = tfidf_4.fit_transform(modelo4['overview_clean'] + ' ' + modelo4['genres'] + ' ' + modelo4['director'])

In [305]:
# Cantidad de registros y columnas en tfidf_matriz_4
tfidf_matriz_4.shape

(45346, 1084451)

In [306]:
def get_recomendacion_m4(titulo):
  # Se crea un objeto 'indices' que mapea los títulos de las películas a sus índices correspondientes en el DataFrame 'modelo4'
  indices = pd.Series(modelo4.index, index=modelo4['title']).drop_duplicates()
  # Se busca el índice de la película de entrada utilizando el objeto 'indices'
  idx = pd.Series(indices[titulo]) if titulo in indices else None
  # Si la película de entrada no se encuentra en el DataFrame, se devuelve un mensaje de error
  if idx is None:
    return "Película no encontrada"
  # Si el título de la película está duplicado, se devuelve el índice de la primera aparición del título en el DataFrame
  if modelo4.duplicated(['title']).any():
    primer_idx = modelo4[modelo4['title'] == titulo].index[0]
    if not idx.equals(pd.Series(primer_idx)):
      idx = pd.Series(primer_idx)
  # Se calcula la similitud coseno entre la película de entrada y todas las demás películas en la matriz de características
  simil = sorted(enumerate(cosine_similarity(tfidf_matriz_4[idx], tfidf_matriz_4).flatten()), key=lambda x: x[1], reverse=True)[1:6]
  # Se obtienen los títulos de las películas más similares utilizando el índice de cada película
  recomendaciones = modelo4.iloc[[i[0] for i in simil], :]['title'].tolist()
  # Se devuelve la lista de títulos de las películas recomendadas
  return recomendaciones

In [307]:
# Prueba #1
get_recomendacion_m4('The Dark Knight Rises')

['The Dark Knight',
 'Batman Forever',
 'Batman Returns',
 'Batman',
 'Batman Begins']

In [308]:
# Prueba #2
get_recomendacion_m4('The Avengers')

['The Work and the Glory',
 'The Right Kind of Wrong',
 'Benny & Joon',
 'Diabolique',
 "Sir Arne's Treasure"]

In [309]:
# Prueba #3 (Película que ha sido lanzada varias veces bajo el mismo nombre)
get_recomendacion_m4('Wonder Woman')

['DC Showcase: Catwoman',
 'Green Lantern: First Flight',
 'Wide Eyed and Legless',
 'Green Lantern: Emerald Knights',
 'Superman: Doomsday']

In [310]:
# Prueba #4 (Película que ha sido lanzada varias veces bajo el mismo nombre)
get_recomendacion_m4('Titanic')

['Ghosts of the Abyss',
 'Grantham and Rose',
 'Titanic',
 'The Legend of 1900',
 'Deadly Voyage']

In [311]:
# Prueba #5
get_recomendacion_m4('Star Wars')

['The Empire Strikes Back',
 'The Star Wars Holiday Special',
 'Return of the Jedi',
 'Star Wars: The Force Awakens',
 'Star Wars: Episode I - The Phantom Menace']

In [312]:
# Prueba #6
get_recomendacion_m4('The Hunger Games')

['The Hunger Games: Catching Fire',
 'The Hunger Games: Mockingjay - Part 2',
 'The Starving Games',
 'The Hungover Games',
 'The Hunger Games: Mockingjay - Part 1']

In [313]:
modelo4 = get_recomendacion_m4('The Hunger Games')

### Modelo 5

title, overview, genres, director, actors

In [314]:
# Seleccionando sólo las columnas necesarias para el modelo
modelo5 = df[['title', 'actores', 'overview_clean', 'genres', 'director']].copy()
# Se realiza un preprocesamiento en la columna "genres" para separar los géneros y convertirlos en palabras individuales
modelo5['genres'] = modelo5['genres'].fillna('').apply(lambda x: ' '.join(x.replace(',', ' ').replace('-', '').lower().split()))
# Se realiza un preprocesamiento en la columna "director" para separar los géneros y convertirlos en palabras individuales
modelo5['director'] = modelo5['director'].fillna('').apply(lambda x: ' '.join(x.replace(',', ' ').replace('-', '').lower().split()))
# Se realiza un preprocesamiento en la columna "actors" para separar los géneros y convertirlos en palabras individuales
modelo5['actores'] = modelo5['actores'].fillna('').apply(lambda x: ' '.join(x.replace(',', ' ').replace('-', '').lower().split()))
# Se realiza un preprocesamiento en la columna "keywords" para separar los géneros y convertirlos en palabras individuales
# modelo5['keywords'] = modelo5['keywords'].fillna('').apply(lambda x: ' '.join(x.replace(',', ' ').replace('-', '').lower().split()))

In [315]:
# Se crea una instancia de la clase TfidfVectorizer con los parámetros deseados
tfidf_5 = TfidfVectorizer(stop_words="english", ngram_range=(1, 2))
# Aplicar la transformación TF-IDF al texto contenido en las columnas "overview_clean", "genres", "actors" y "director" del dataframe 'modelo5'
tfidf_matriz_5 = tfidf_5.fit_transform(modelo5['overview_clean'] + ' ' + modelo5['genres'] + ' ' + modelo5['director'] + ' ' + modelo5['actores'])

In [316]:
# Cantidad de registros y columnas en tfidf_matriz_5
tfidf_matriz_5.shape

(45346, 1865677)

In [317]:
def get_recomendacion_m5(titulo):
  # Se crea un objeto 'indices' que mapea los títulos de las películas a sus índices correspondientes en el DataFrame 'modelo5'
  indices = pd.Series(modelo5.index, index=modelo5['title']).drop_duplicates()
  # Se busca el índice de la película de entrada utilizando el objeto 'indices'
  idx = pd.Series(indices[titulo]) if titulo in indices else None
  # Si la película de entrada no se encuentra en el DataFrame, se devuelve un mensaje de error
  if idx is None:
    return "Película no encontrada"
  # Si el título de la película está duplicado, se devuelve el índice de la primera aparición del título en el DataFrame
  if modelo5.duplicated(['title']).any():
    primer_idx = modelo5[modelo5['title'] == titulo].index[0]
    if not idx.equals(pd.Series(primer_idx)):
      idx = pd.Series(primer_idx)
  # Se calcula la similitud coseno entre la película de entrada y todas las demás películas en la matriz de características
  simil = sorted(enumerate(cosine_similarity(tfidf_matriz_5[idx], tfidf_matriz_5).flatten()), key=lambda x: x[1], reverse=True)[1:6]
  # Se obtienen los títulos de las películas más similares utilizando el índice de cada película
  recomendaciones = modelo5.iloc[[i[0] for i in simil], :]['title'].tolist()
  # Se devuelve la lista de títulos de las películas recomendadas
  return recomendaciones

In [318]:
# Prueba #1
get_recomendacion_m5('The Dark Knight Rises')

['The Dark Knight',
 'Batman Begins',
 'Interstellar',
 'The Wolf of Wall Street',
 'Inception']

In [319]:
# Prueba #2
get_recomendacion_m5('The Avengers')

['The Work and the Glory',
 'Hysterical Blindness',
 'Eddie Izzard: Unrepeatable',
 'Eddie Izzard: Circle',
 'The Frightened City']

In [320]:
# Prueba #3 (Película que ha sido lanzada varias veces bajo el mismo nombre)
get_recomendacion_m5('Wonder Woman')

['DC Showcase: Catwoman',
 'Superman/Batman: Apocalypse',
 'Green Lantern: Emerald Knights',
 'Green Lantern: First Flight',
 'Superman: Doomsday']

In [321]:
# Prueba #4 (Película que ha sido lanzada varias veces bajo el mismo nombre)
get_recomendacion_m5('Titanic')

['Aliens of the Deep',
 'Titanic: The Final Word with James Cameron',
 'Ghosts of the Abyss',
 'The Bounty',
 'The Departed']

In [322]:
# Prueba #5
get_recomendacion_m5('Star Wars')

['The Empire Strikes Back',
 'The Star Wars Holiday Special',
 'Return of the Jedi',
 'Empire of Dreams: The Story of the Star Wars Trilogy',
 'Elstree 1976']

In [323]:
# Prueba #6
get_recomendacion_m5('The Hunger Games')

['The Hunger Games: Catching Fire',
 'The Hunger Games: Mockingjay - Part 2',
 'The Hunger Games: Mockingjay - Part 1',
 "Buster's Mal Heart",
 'Quarantine 2: Terminal']

## Sugerencia de títulos mal escritos

In [324]:
# Seleccionando sólo las columnas necesarias para el modelo
modelo_sug = df[['title', 'actores', 'keywords', 'genres', 'director']].copy()
# Se realiza un preprocesamiento en la columna "genres" para separar los géneros y convertirlos en palabras individuales
modelo_sug['genres'] = modelo_sug['genres'].fillna('').apply(lambda x: ' '.join(x.replace(',', ' ').replace('-', '').lower().split()))
# Se realiza un preprocesamiento en la columna "director" para separar los géneros y convertirlos en palabras individuales
modelo_sug['director'] = modelo_sug['director'].fillna('').apply(lambda x: ' '.join(x.replace(',', ' ').replace('-', '').lower().split()))
# Se realiza un preprocesamiento en la columna "actors" para separar los géneros y convertirlos en palabras individuales
modelo_sug['actores'] = modelo_sug['actores'].fillna('').apply(lambda x: ' '.join(x.replace(',', ' ').replace('-', '').lower().split()))
# Se realiza un preprocesamiento en la columna "keywords" para separar los géneros y convertirlos en palabras individuales
modelo_sug['keywords'] = modelo_sug['keywords'].fillna('').apply(lambda x: ' '.join(x.replace(',', ' ').replace('-', '').lower().split()))

In [325]:
# Se crea una instancia de la clase TfidfVectorizer con los parámetros deseados
tfidf_sug = TfidfVectorizer(stop_words="english", ngram_range=(1, 2))
# Aplicar la transformación TF-IDF al texto contenido en las columnas "keywords", "genres", "actores" y "director" del dataframe 'modelo_sug'
tfidf_matriz_sug = tfidf_sug.fit_transform(modelo_sug['keywords'] + ' ' + modelo_sug['genres'] + ' ' + modelo_sug['director'] + ' ' + modelo_sug['actores'])

In [326]:
def suggest_movie_title(movie_title):
  # Obtener una lista de todos los títulos de películas disponibles
  available_movie_titles = modelo_sug['title'].tolist()
  # Encontrar las coincidencias más cercanas al título ingresado
  matches = process.extract(movie_title, available_movie_titles, limit=5)
  # Obtener las mejores coincidencias y sus puntajes de similitud
  best_matches = [match[0] for match in matches]
  best_match_scores = [match[1] for match in matches]
  # Verificar si hay una coincidencia lo suficientemente cercana
  if best_match_scores[0] >= 70:
    suggestion = best_matches[0]
    return f"Quizás quisiste decir: {suggestion}"
  else:
    return "No se encontraron coincidencias cercanas."

In [327]:
def get_recomendacion_msug(titulo):
  # Se crea un objeto 'indices' que mapea los títulos de las películas a sus índices correspondientes en el DataFrame 'modelo_sug'
  indices = pd.Series(modelo_sug.index, index=modelo_sug['title']).drop_duplicates()
  # Se busca el índice de la película de entrada utilizando el objeto 'indices'
  idx = pd.Series(indices[titulo]) if titulo in indices else None
  # Si la película de entrada no se encuentra en el DataFrame, se verifica si hay una sugerencia de título
  if idx is None:
    suggestion = suggest_movie_title(titulo)
    if suggestion != "No se encontraron coincidencias cercanas.":
      return suggestion
    else:
      return "Película no encontrada"
  # Si el título de la película está duplicado, se devuelve el índice de la primera aparición del título en el DataFrame
  if modelo_sug.duplicated(['title']).any():
    primer_idx = modelo_sug[modelo_sug['title'] == titulo].index[0]
    if not idx.equals(pd.Series(primer_idx)):
      idx = pd.Series(primer_idx)
  # Se calcula la similitud coseno entre la película de entrada y todas las demás películas en la matriz de características
  simil = sorted(enumerate(cosine_similarity(tfidf_matriz_sug[idx], tfidf_matriz_sug).flatten()), key=lambda x: x[1], reverse=True)[1:6]
  # Se obtienen los títulos de las películas más similares utilizando el índice de cada película
  recomendaciones = modelo_sug.iloc[[i[0] for i in simil], :]['title'].tolist()
  # Se devuelve la lista de títulos de las películas recomendadas
  return recomendaciones

In [328]:
# Prueba #1
get_recomendacion_msug('The Dark Knight Rises')

['The Dark Knight',
 'Batman Begins',
 'Interstellar',
 'The Wolf of Wall Street',
 'Inception']

In [329]:
# Prueba #1 Título mal escrito
get_recomendacion_msug('The Dark Knigth Rises')

'Quizás quisiste decir: The Dark Knight Rises'

In [330]:
# Prueba #2
get_recomendacion_msug('Star Wars')

['The Empire Strikes Back',
 'The Star Wars Holiday Special',
 'Return of the Jedi',
 'Empire of Dreams: The Story of the Star Wars Trilogy',
 'Elstree 1976']

In [331]:
# Prueba #2 Título mal escrito
get_recomendacion_msug('Stars Wars')

'Quizás quisiste decir: Star Wars'

# Resultados de recomendación basado en contenido

## Pelicula: The Dark Knight Rises

In [333]:
# Imprimir las primeras 5 filas del Dataframe "merged"
df[['title', 'overview_clean']].head()

,title,overview_clean
0,Toy Story,led by woody andys toy live happily in his roo...
1,Jumanji,when sibling judy and peter discover an enchan...
2,Grumpier Old Men,a family wedding reignites the ancient feud be...
3,Waiting to Exhale,cheated on mistreated and stepped on the woman...
4,Father of the Bride Part II,just when george bank ha recovered from his da...


In [335]:
# Se exporta las columnas necesarias para el Sistema de Recomendación
modelo = df[['title', 'overview_clean']]
modelo.to_parquet('movies_recomendaciones.parquet')